In [ ]:
import numpy as np
import pandas as pd

# just for plots
import plotly.express as px

from sktime.datatypes import get_examples
from sktime.transformations.hierarchical.reconcile import reconciler

# https://otexts.com/fpp3/hierarchical.html
# https://github.com/robjhyndman/reconciliation_review_talk/blob/main/10years_reconciliation.pdf

# Hierarchical dataset

In [ ]:
df = get_examples(mtype="pd_multiindex_hier", as_scitype="Hierarchical")
df = df[0]

df

## Aggregate Hierarchy

In [ ]:
def aggregate_hierarchy(df_hier, flatten_single_levels=True):
    """From hierarchical mtype get the full aggregate hierarchy before forecasting"""

    if df_hier.index.nlevels >= 2:
        hier_names = list(df_hier.index.names)

        # top level
        # remove aggregations that only have one level from below
        if flatten_single_levels:
            single_df = df_hier.groupby(["timepoints"]).count()
            mask1 = (
                single_df[(single_df > 1).all(1)]
                .index.get_level_values("timepoints")
                .unique()
            )
            mask1 = df_hier.index.get_level_values("timepoints").isin(mask1)
            top = df_hier.loc[mask1].groupby(level=["timepoints"]).sum()
        else:
            top = df_hier.loc[mask1].groupby(level=["timepoints"]).sum()

        ind_names = list(set(hier_names).difference(["timepoints"]))
        for i in ind_names:
            top[i] = "__total"

        top = top.set_index(ind_names, append=True).reorder_levels(hier_names)

        df_out = pd.concat([top, df_hier])

        # if we have a hierarchy with mid levels
        if len(hier_names) > 2:
            for i in range(len(hier_names) - 2):
                # list of levels to aggregate
                agg_levels = hier_names[0 : (i + 1)]
                agg_levels.append("timepoints")

                # remove aggregations that only have one level from below
                if flatten_single_levels:
                    single_df = df_hier.groupby(level=agg_levels).count()
                    # get index masks
                    masks = []
                    for i in agg_levels:
                        m1 = (
                            single_df[(single_df > 1).all(1)]
                            .index.get_level_values(i)
                            .unique()
                        )
                        m1 = df_hier.index.get_level_values(i).isin(m1)
                        masks.append(m1)
                    mid = (
                        df_hier.loc[np.logical_and.reduce(masks)]
                        .groupby(level=agg_levels)
                        .sum()
                    )
                else:
                    mid = df_hier.groupby(level=agg_levels).sum()

                # now fill in index
                ind_names = list(set(hier_names).difference(agg_levels))
                for j in ind_names:
                    mid[j] = "__total"
                # set back in index
                mid = mid.set_index(ind_names, append=True).reorder_levels(hier_names)
                df_out = pd.concat([df_out, mid])

        df_out.sort_index(inplace=True)
        return df_out
    else:
        return df_hier

Now we have the full forecasting dataset

In [ ]:
aggregate_hierarchy(df)

Let's test with bottom levels that span two nodes

- i.e. mid levels that are only present at a subset of bottom nodes

In [ ]:
cols = ["foo", "foo2", "bar", "timepoints"] + [f"var_{i}" for i in range(2)]

Xlist = [
    pd.DataFrame(
        [["a", "a1", 0, 0, 1, 4], ["a", "a1", 0, 1, 2, 5], ["a", "a1", 0, 2, 3, 6]],
        columns=cols,
    ),
    pd.DataFrame(
        [["a", "a1", 1, 0, 1, 4], ["a", "a1", 1, 1, 2, 55], ["a", "a1", 1, 2, 3, 6]],
        columns=cols,
    ),
    pd.DataFrame(
        [["a", "a2", 2, 0, 1, 42], ["a", "a2", 2, 1, 2, 5], ["a", "a2", 2, 2, 3, 6]],
        columns=cols,
    ),
    pd.DataFrame(
        [["b", "b1", 0, 0, 1, 4], ["b", "b1", 0, 1, 2, 5], ["b", "b1", 0, 2, 3, 6]],
        columns=cols,
    ),
    pd.DataFrame(
        [["b", "b2", 1, 0, 1, 4], ["b", "b2", 1, 1, 2, 55], ["b", "b2", 1, 2, 3, 6]],
        columns=cols,
    ),
    pd.DataFrame(
        [["b", "b2", 2, 0, 1, 42], ["b", "b2", 2, 1, 2, 5], ["b", "b2", 2, 2, 3, 6]],
        columns=cols,
    ),
]
X = pd.concat(Xlist)
X = X.set_index(["foo", "foo2", "bar", "timepoints"])

X

Note flatten single levels is the default option

- see that `(a, a2, 2, *)` and `(b, b1, 0, *)` don't contain `__total`

In [ ]:
aggregate_hierarchy(X, flatten_single_levels=True)

# Forecasting Example

Let's generate a hierarchical dataset similar to the last example from the flights dataset

- Generate dataset
- Generate full hierarchy
- Forecast each level
- Reconcile

## Generate Dataset

In [ ]:
from sktime.datasets import load_airline
from sktime.utils.plotting import plot_series

In [ ]:
zone1 = load_airline()

zone1

In [ ]:
# plotting for visualization
plot_series(
    zone1,
    10 + zone1 * 5,
    -50 + zone1 * 0.9,
    zone1 ** 1.5,
    -20 + 10 * zone1,
    10 + (10 * zone1) + (0.05 * (zone1 ** 2)),
    labels=["zone1", "zone2", "zone3", "zone4", "zone5", "zone6"],
)

In [ ]:
df = pd.DataFrame(zone1, index=zone1.index).rename(
    columns={"Number of airline passengers": "zone1"}
)

df["zone2"] = 10 + zone1 * 5
df["zone3"] = zone1 * 0.9 - 50
df["zone4"] = zone1 ** 1.5
df["zone5"] = zone1 * 10 - 500
df["zone6"] = 10 + (10 * zone1) + (0.05 * (zone1 ** 2))

df = (
    df.melt(ignore_index=False)
    .set_index(["variable", df.melt(ignore_index=False).index])
    .rename_axis(["airport", "timepoints"], axis=0)
    .rename(columns={"value": "passengers"})
)

# df['country'] = "USA"
df.loc[
    df.index.get_level_values(level="airport").isin(["zone1", "zone2", "zone3"]),
    "state",
] = "CA"
df.loc[
    df.index.get_level_values(level="airport").isin(["zone1", "zone2"]), "city"
] = "LA"
df.loc[df.index.get_level_values(level="airport").isin(["zone3"]), "city"] = "SF"


df.loc[
    df.index.get_level_values(level="airport").isin(["zone4", "zone5", "zone6"]),
    "state",
] = "NY"
df.loc[
    df.index.get_level_values(level="airport").isin(["zone4", "zone5"]), "city"
] = "NYC"
df.loc[df.index.get_level_values(level="airport").isin(["zone6"]), "city"] = "BF"

df = df.set_index(["state", "city", df.index])
df


# df.droplevel(level=-1).index.unique()

## Generate full hierarchy

In [ ]:
df_fh = aggregate_hierarchy(df, flatten_single_levels=True)

df_fh

## Forecast each level

here we will forecast each unique level outside `timepoints`

In [ ]:
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# from sktime.forecasting.model_selection import temporal_train_test_split
# from sktime.performance_metrics.forecasting import mean_absolute_percentage_error

In [ ]:
model_ids = df_fh.droplevel(level="timepoints").index.unique()

model_ids

In [ ]:
# Now set up loop for forecasting
# # for i in model_ids:
# mods = {}
# prds = {}

# for i in model_ids:
#     # i = model_ids[0]
#     y_train, y_test = temporal_train_test_split(df_fh.loc[i], test_size=36)
#     fh = ForecastingHorizon(y_test.index, is_relative=False)
#     forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
#     mods[i] = forecaster.fit(y_train)
#     prds[i] = forecaster.predict(fh)
#     # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
#     print(i)
#     print(mean_absolute_percentage_error(y_test, prds[i], symmetric=True))
# prds = (
#     pd.concat(prds)
#     .rename_axis(df_fh.index.names, axis=0)
#     .rename(columns={"passengers": "y_pred"})
# )

# # join with meas
# prds = pd.concat([prds, df_fh], axis=1, join="inner").rename(
#     columns={"passengers": "y_true"}
# )

In [ ]:
# # for i in model_ids:
# mods = {}
# prds = {}

# for i in model_ids:
# i = model_ids[0]
# y_train, y_test = temporal_train_test_split(df_fh, test_size=36)
fh = ForecastingHorizon([1, 2, 3, 4, 5, 6], is_relative=True)
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
mods = forecaster.fit(df_fh)
prds = forecaster.predict(fh)
# plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
# print(i)
# print(mean_absolute_percentage_error(df_fh, prds, symmetric=True))
prds

# prds.index = prds.index.rename(['state', 'city', 'airport', 'timepoints'])

# prds

## Reconcile - Bottom Up

Bottom up is easy we just sum the bottome levels much like aggregate function.

But we want it to be compatible with other methods which go like
    
    - get y 'base' forecasts for all series (previous section)
    - get S matrix from df index (defined by hierarchy structure)
    - get G matrix for recon (defined by recon method)
    - reconcile forecasts - SGy (all methods)


In [ ]:
transformer = reconciler(method="bu")

fitted_transfrom = transformer.fit(X=prds[["passengers"]])

fitted_transfrom.s_matrix

# https://stackoverflow.com/questions/54307300/what-causes-indexing-past-lexsort-depth-warning-in-pandas

In [ ]:
fitted_transfrom.g_matrix

In [ ]:
prds["y_recon_bu"] = fitted_transfrom.transform(X=prds[["passengers"]])

prds

This seems to work fine

In [ ]:
prds.loc[prds.index.get_level_values(level=-1) == "1961-01"]

## OLS reconciliation

    - Now all we need is the new g_matrix method

In [ ]:
transformer_ols = reconciler(method="ols")

fitted_transfrom_ols = transformer_ols.fit(X=prds[["passengers"]])

fitted_transfrom_ols.g_matrix

In [ ]:
prds["y_recon_ols"] = fitted_transfrom_ols.transform(X=prds[["passengers"]])

prds

This seems to work fine as well

    - note the bottom level forecasts have now changed as well

In [ ]:
prds.loc[prds.index.get_level_values(level=-1) == "1961-01"]

## WLS structural reconciliation

    - Now all we need is the new g_matrix method

In [ ]:
transformer_wls = reconciler(method="wls_str")

fitted_transfrom_wls = transformer_wls._fit(X=prds[["passengers"]])

fitted_transfrom_wls.g_matrix

In [ ]:
prds["y_recon_wls"] = fitted_transfrom_wls._transform(X=prds[["passengers"]])

prds

In [ ]:
prds.loc[prds.index.get_level_values(level=-1) == "1961-01"]

# Test different hierarchy set ups

- set up work for different hierarchies
- first we need a data generating function

In [ ]:
def bottom_hier_datagen(
    no_levels=3,
    no_bottom_nodes=6,
    intercept_max=20,
    coef_1_max=20,
    coef_2_max=0.1,
    power_2=[0.5, 1, 1.5, 2],
):

    if no_levels > no_bottom_nodes:
        raise ValueError("no_levels should be less than no_bottom_nodes")

    base_ts = load_airline()
    df = pd.DataFrame(base_ts, index=base_ts.index)
    df.index.rename(None, inplace=True)

    if no_levels == 0:
        df.columns = ["passengers"]
        df.index.rename("timepoints", inplace=True)
        return df
    else:

        df.columns = ["l1_node01"]

        intercept = np.arange(0, intercept_max, 0.01)
        coef_1 = np.arange(0, coef_1_max, 0.01)
        coef_2 = np.arange(0, coef_2_max, 0.01)

        node_lookup = pd.DataFrame(
            ["l1_node" + f"{x:02d}" for x in range(1, no_bottom_nodes + 1)]
        )
        node_lookup.columns = ["l1_agg"]

        if no_levels >= 2:

            for i in range(2, no_levels + 1):
                node_lookup["l" + str(i) + "_agg"] = node_lookup.groupby(
                    ["l" + str(i - 1) + "_agg"]
                )["l1_agg"].transform(
                    lambda x: "l"
                    + str(i)
                    + "_node"
                    + f"{int(np.sort(np.random.choice(np.arange(1, np.floor(len(node_lookup.index)/i)+1, 1), size=1))):02d}"  # noqa from flake8 E501
                )

        node_lookup = node_lookup.set_index("l1_agg", drop=True)

        for i in range(2, no_bottom_nodes + 1):
            df["l1_node" + f"{i:02d}"] = (
                np.random.choice(intercept, size=1)
                + np.random.choice(coef_1, size=1) * df["l1_node01"]
                + (
                    np.random.choice(coef_2, size=1)
                    * (df["l1_node01"] ** np.random.choice(power_2, size=1))
                )
            )

        df = (
            df.melt(ignore_index=False)
            .reset_index(drop=False)
            .rename(
                columns={
                    "variable": "l1_agg",
                    "index": "timepoints",
                    "value": "passengers",
                }
            )
        )

        df = pd.merge(left=df, right=node_lookup.reset_index(), on="l1_agg")
        df = df[df.columns.sort_values(ascending=True)]

        df_newindex = ["l" + str(x) + "_agg" for x in range(1, no_levels + 1)][::-1]
        df_newindex.append("timepoints")

        df = df.set_index(df_newindex)
        df.sort_index(inplace=True)

        return df


np.random.seed(4)
df = bottom_hier_datagen(no_bottom_nodes=20, no_levels=3)

df["node_id"] = ["_".join(x) for x in list(df.droplevel(-1).index)]

fig = px.scatter(
    data_frame=df,
    x=df.index.get_level_values(-1).to_timestamp(),
    y="passengers",
    color="node_id",
)
fig.update_traces(marker={"size": 5})
fig.show()

In [ ]:
df = aggregate_hierarchy(df, flatten_single_levels=True)
df["node_id"] = ["_".join(x) for x in list(df.droplevel(-1).index)]

fig = px.scatter(
    data_frame=df,
    x=df.index.get_level_values(-1).to_timestamp(),
    y="passengers",
    color="node_id",
)
fig.update_traces(marker={"size": 5})
fig.show()

In [ ]:
np.random.seed(4)
df = bottom_hier_datagen(no_bottom_nodes=15, no_levels=4)

df = aggregate_hierarchy(df, flatten_single_levels=True)
df

fh = ForecastingHorizon([1, 2, 3, 4, 5, 6], is_relative=True)
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
mods = forecaster.fit(df[["passengers"]])
prds = forecaster.predict(fh)
prds

transformer = reconciler(method="bu")
fitted_transfrom = transformer.fit(X=prds[["passengers"]])
prds["y_recon_bu"] = fitted_transfrom.transform(X=prds[["passengers"]])


transformer = reconciler(method="ols")
fitted_transfrom = transformer.fit(X=prds[["passengers"]])
prds["y_recon_ols"] = fitted_transfrom.transform(X=prds[["passengers"]])


transformer = reconciler(method="wls_str")
fitted_transfrom = transformer.fit(X=prds[["passengers"]])
prds["y_recon_wls"] = fitted_transfrom.transform(X=prds[["passengers"]])


prds.loc[prds.index.get_level_values(level=-1) == "1961-01"]
# prds.loc[prds.index.get_level_values(level=-1) == "1961-01"]
# .drop_duplicates(keep = "last")

In [ ]:
test_df = prds.loc[
    prds.index.get_level_values(level=-2) != "__total", prds.columns[1:4]
].copy()
test_df.index.set_names("timepoints", level=-1, inplace=True)

test_df = aggregate_hierarchy(test_df, flatten_single_levels=True)
# test_df = agg_df.fit_transform(X=test_df)
# test_df.index.names = ["state", "city", "airport", None]
# test_df.equals(prds[prds.columns[1:4]])
(test_df - prds[prds.columns[1:4]]).apply(lambda x: x.round(6).unique())

In [ ]:
np.random.seed(4)
df = bottom_hier_datagen(base_ts=zone1, no_bottom_nodes=20, no_levels=2)
df.droplevel(-1).index.unique()

# Tests And checks

ok for the aggregation check that

    -    test that "__total" is named in the first index *
    -    that the method is recognised
    -    test that the bottom levels are unique *

In [ ]:
# should fail
transformer = reconciler(method="bux")
fitted_transfrom = transformer.fit(X=prds[["passengers"]])

In [ ]:
# should fail
transformer = reconciler(method="bu")
fitted_transfrom = transformer.fit(X=X[["var_1"]])

In [ ]:
# non-unique bottom level names
df = get_examples(mtype="pd_multiindex_hier", as_scitype="Hierarchical")
df = df[0]
df = aggregate_hierarchy(df)
df

In [ ]:
transformer = reconciler(method="bu")
fitted_transfrom = transformer.fit(X=df[["var_1"]])

In [ ]:
# for i in model_ids:
#     # print(i)
#     # print(
#     #     mean_absolute_percentage_error(
#     #         prds.loc[i, "y_true"], prds.loc[i, "y_pred"], symmetric=True
#     #     )
#     # )
#     # print(
#     #     mean_absolute_percentage_error(
#     #         prds.loc[i, "y_true"], prds.loc[i, "y_reco_bu"], symmetric=True
#     #     )
#     # )
#     # print(
#     #     mean_absolute_percentage_error(
#     #         prds.loc[i, "y_true"], prds.loc[i, "y_reco_ols"], symmetric=True
#     #     )
#     # )
#     plot_series(
#         prds.loc[i, 'y_true'],
#         prds.loc[i, 'y_pred'],
#         prds.loc[i, 'y_reco_bu'],
#         prds.loc[i, 'y_reco_ols'],
#         labels=["y_test", "y_pred", "y_pred_bu", "y_pred_ols"],
#     )

# index = ("LA", "zone1", "sth")
# "__".join(index)

So we could maybe work it like this


class (panel_forecaster)
    
    - fit
    - predict
    - train_test_temporal split?
    - list of model specs

class hierarchical_forecaster(panel_forecaster)
    
    Includes the aggregated levels for the panel.
    
    Inherits methods from above and adds g matrix methods that need information from model fits/original data

    - get_g_matrix_wlsvar
    - get_g_matrix_mint
    - get_g_matrix_mint_shrink
    - get_g_matrix_topdown
    - predict generates multiindex

class reconcile(Transfromer, predictions: multi-index with '__total' present, method = "bu"):

    Inherets transfromer methods? and includes reconciliation methods that don't depend on historic/residual data.

    Checks we have predicttions from hierarchical forecaster then

    - fit
    - predict, i.e. reconcile from this notebook
    - get_s_matrix
    - get_g_matrix_bu
    - get_g_matrix_ols
    - get_g_matrix_wlsstr
